# Sensor FOV and Swath Calculator
### Copyright 2018, Augmntr Inc.
### License permission granted to Ron Chapple / Aerial Filmworks and associated companies only

The unit of AGL is unspecified, so all results will be in that same unit

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# All the imports
from ipywidgets import interact, interact_manual
from IPython.core.display import display, HTML
import ipywidgets as widgets
import math

import numpy as np
                                                                                                                                                           
import matplotlib
import matplotlib.pyplot as plt


In [3]:
# Adjust the display to full width
display(HTML("<style>.container { width:100% !important; }</style>"))

# Hide all that code
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to show the raw code."></form>''')


In [4]:
# Input widgets
style = {'description_width': 'initial'}

i_hpixels = widgets.BoundedIntText( min = 1, max = 999999999, value = 1280, description='Horizontal Pixels:', style=style )
i_vpixels = widgets.BoundedIntText( min = 1, max = 999999999, value = 720,  description='Vertical Pixels:', style=style )
i_pitch = widgets.BoundedFloatText( min=0.01, max=9999999, value=10, description='Pixel Pitch (µm):', style=style )
i_fl = widgets.BoundedFloatText( min = 1, max = 10000, value = 50, description='Lens Focal Length (mm):', style=style )
i_agl = widgets.BoundedFloatText( min = 0.001, max = 9999999999999, value = 450., description='AGL (m):', style=style )
i_angle = widgets.BoundedFloatText( min = 0, max = 90, value = 45, description='Lookdown Angle (°):', style=style )

In [11]:
def GroundIntercept(rotmatrix, offset, disttoground ):
    rot = rotmatrix * offset
    mag = math.sqrt((rot[0] ** 2) + (rot[1] ** 2) + (rot[2] ** 2))
    alpha = ((rot[0])/mag).item(0)
    beta = ((rot[1])/mag).item(0)
    gamma = ((rot[2])/mag).item(0)
    x = (disttoground * alpha)/gamma 
    y = (disttoground * beta)/gamma 
    z = disttoground
    return (x,y,z), beta

In [41]:
def PrintAndCalc(pixelpitch_um, hpixels, vpixels, fl_mm, agl_m, lookdown_deg):
    
    lookdown_rad = -1 * math.radians(lookdown_deg)
    
    # CALCULATING SENSOR CHARACTERISTICS
    fl_m = fl_mm / 1000
    
    pixelpitch_m = pixelpitch_um / 1000000
    
    ifov_rad = math.atan(pixelpitch_m/fl_m)
    ifov_urad = ifov_rad * 1000000
    
    #gsd_cm = ifov_urad
    
    hfov_rad = ifov_rad * hpixels
    vfov_rad = ifov_rad * vpixels
    hfov_deg = math.degrees(hfov_rad)
    vfov_deg = math.degrees(vfov_rad)
    
    sensor_width_mm = hpixels * pixelpitch_m * 1000
    sensor_height_mm = vpixels * pixelpitch_m * 1000
    sensor_diag_mm = math.sqrt(sensor_width_mm ** 2 + sensor_height_mm ** 2)
    
    optical_format_mm = sensor_diag_mm * 1.5
    optical_format_inches = optical_format_mm * 0.0393701

    print(f"Lookdown Angle:         {lookdown_deg:.2f}°")
    print(f"Sensor WxH:             {sensor_width_mm:.2f}mm x {sensor_height_mm:.2f}mm" )
    print(f"Sensor Diagonal:        {sensor_diag_mm:.2f}mm")
    print(f"Sensor Optical Format:  {optical_format_mm:.2f}mm, {optical_format_inches:.2f}\"")
    print(f"IFOV:                   {ifov_urad:.2f}µrad")
    print(f"FOV:                    {hfov_deg:.2f}° x {vfov_deg:.2f}°")
    print("")
    
    #SETTING TOTAL TRACK DISTANCE FROM IMAGE FOCUS TO GROUND AS THE LEVER ARM FOR ROTATION.  FOCUS POINT ASSUMED TO BE THE CENTER POINT FOR ALL ROTATIONS
    
    disttoground = -1 * (agl_m + fl_m) 
    
    # C = Center, T = Top, B = Bottom, R = Right, L = Left
    
    # Setting initial FPA center to look at flight forward
    sensC = np.matrix([[0],[fl_mm],[0]])
    sensTR = np.matrix([[(sensor_width_mm/-2)],[fl_mm],[(sensor_height_mm/2)]])
    sensBR = np.matrix([[(sensor_width_mm/-2)],[fl_mm],[(sensor_height_mm/-2)]])
    sensBL = np.matrix([[(sensor_width_mm/2)],[fl_mm],[(sensor_height_mm/-2)]])
    sensTL = np.matrix([[(sensor_width_mm/2)],[fl_mm],[(sensor_height_mm/2)]])
    sensR = np.matrix([[(sensor_width_mm/-2)],[fl_mm],[0]])
    sensB = np.matrix([[0],[fl_mm],[(sensor_height_mm/-2)]])
    sensL = np.matrix([[(sensor_width_mm/2)],[fl_mm],[0]])
    sensT = np.matrix([[0],[fl_mm],[(sensor_height_mm/2)]])
   
    ###
    # Calculating the master rotation matrix based on the lookdown angle
    rotmatrix = np.matrix([[1, 0, 0], [0, math.cos(lookdown_rad), (-1 * math.sin(lookdown_rad))], [0, math.sin(lookdown_rad), math.cos(lookdown_rad)]])
    
    # Central ray
    groundC,_ = GroundIntercept( rotmatrix, sensC, disttoground )
      
    # Corners
    groundTR,_ = GroundIntercept( rotmatrix, sensTR, disttoground )
    groundBR,_ = GroundIntercept( rotmatrix, sensBR, disttoground )
    groundBL,_ = GroundIntercept( rotmatrix, sensBL, disttoground )
    groundTL,_ = GroundIntercept( rotmatrix, sensTL, disttoground )
    
    # Mids
    groundR,_ = GroundIntercept( rotmatrix, sensR, disttoground )
    groundB,_ = GroundIntercept( rotmatrix, sensB, disttoground )
    groundL,_ = GroundIntercept( rotmatrix, sensL, disttoground )
    groundT, angleT = GroundIntercept( rotmatrix, sensT, disttoground )
    
    # Check for successful ground intercept before calling plots
    vdownlook = ((-1*lookdown_rad) - (0.5*vfov_rad))
    hdownlook = (0.5*hfov_rad)
            
    if vdownlook <= 0 or hdownlook <= 0 :
        print('Not all points intercept ground - Sensor FOV is >180 degrees in at least one axis')
    else:    
        draw_plots( angleT, lookdown_rad, hfov_rad, vfov_rad, agl_m, groundC, groundBL, groundBR, groundTL, groundTR, groundR, groundT, groundL,groundB )

        
    #MAKE A NEW FUNCTION HERE JUST FOR SANITY THAT HITS THE 1/2 FOV VECTOR WITH THE ROTATION MATRIX TO DETERMINE THE REAL ANGLE OF INTERCEPTION DANGLE
def draw_plots( betarotmidT, lookdown_rad, hfov_rad, vfov_rad, agl, center, bl, br, tl, tr, r, t, l, b ):
        
    far_width = abs(tr[0]-tl[0])
    near_width = abs(br[0]-bl[0])
    far_to_near = tr[1]-br[1]
    middle_width = abs(l[0]-r[0])
    front_back_length = abs(t[1]-b[1])

    print(f"Width at Bottom Frame:  {near_width:.1f}m")
    print(f"Width at Mid Frame:     {middle_width:.1f}m")
    print(f"Width at Top Frame:     {far_width:.1f}m")
    print(f"Front to Back Length:   {front_back_length:.1f}m")

    leftlabel = (tl[1]+bl[1])/2
    rightlabel = (tr[1]+br[1])/2
    #print(f'Left Label is {leftlabel}')
    #print(f'Right Label is {rightlabel}')
    
    # draw TopDown plot - TopDown is the y-intercept point from the frame of reference above
    topdown_x=[]
    topdown_y=[]
       
    #topdown_x = The x axis for the displayed graph, but is the y-axis coordinate in the matrix passed to this function
    #topdown_y = The y axis for the displayed graph, but is the x-axis coordinate in the matrix passed to this function
    topdown_x.append( bl[1] )
    topdown_y.append( bl[0] )

    topdown_x.append( br[1] )
    topdown_y.append( br[0] )

    topdown_x.append( tr[1] )
    topdown_y.append( tr[0] )

    topdown_x.append( tl[1] )
    topdown_y.append( tl[0] )

    topdown_x.append( bl[1] )
    topdown_y.append( bl[0] )
    
    plt.title("Topdown View")
    plt.axis('equal')
    plt.plot(topdown_x,topdown_y,  color="green")
    plt.plot([center[1]], [center[0]], marker='o', markersize=3, color="red")

    plt.plot( [r[1],l[1]],[r[0],l[0]], color="blue")

    plt.plot( [b[1],t[1]],[b[0],t[0]], color="yellow")
    
    plt.text(bl[1], 0, f"{near_width:.1f}", va='center', ha='right', rotation='vertical' )
    plt.text(tl[1], 0, f"{far_width:.1f}", va='center', ha='right', rotation='vertical' )
    plt.text(center[1], 0, f"{middle_width:.1f}", va='center', ha='right', rotation='vertical' )
    plt.text(center[1], 0, f" {front_back_length:.1f}", ha='left' )
   
    
    # draw side plot
    side_x=[]
    side_y=[]
    los_x=[]
    los_y=[]
    los_x.append( 0 )
    los_y.append( agl )
    
    los_x.append( center[1] )
    los_y.append( 0 )

    side_x.append( 0 )
    side_y.append( agl )
    
    side_x.append( tl[1] )
    side_y.append( 0 )
    
    side_x.append( bl[1] )
    side_y.append( 0 )
    
    side_x.append( 0 )
    side_y.append( agl )
    
    plt.figure()
    plt.title("Side View")
    plt.axis('equal')
    # plot the swath
    plt.plot(side_x,side_y, color="green")
    plt.plot([center[1]], [0], marker='o', markersize=3, color="red")
    plt.text(center[1], 0, f"{far_to_near:.1f}", va='center', ha='center' )

    # Now calculating info needed to plot the heights
    
    # Finding the interior intersection angle - accomodating for the FOV of the sensor in this step - this midMA ray represents the intersection of the Y-axis as projected on the ground
    
    upperrayintangle_rad = (math.acos(betarotmidT))
    
    # Determining the edgeheight by using the tangent of the angle and the known far_to_near distance
    
    edgeheight = far_to_near * math.tan(upperrayintangle_rad)
    
    # This is logic to define what happens when the sensor looks past nadir on the lower side
    if edgeheight > agl:
        edgeheight = agl
    
    # Find far_to_center to determine the distance of the second, smaller triangle - once we have this can use the tangent relationship again with the same interior angle
    
    far_to_center = tl[1] - center[1]

    # Now we can get the centerheight
    simplelookdown = np.around(lookdown_rad, decimals=6)
        
    if simplelookdown == -1.570796:
        centerheight = agl
    else:
        centerheight = far_to_center * math.tan(upperrayintangle_rad)
    
    print(f"Height at Bottom:       {edgeheight:.1f}m")
    print(f"Height at Center:       {centerheight:.1f}m")

    plt.plot( [bl[1],bl[1]],[0,edgeheight], color="blue")
    plt.plot( [center[1],center[1]],[0,centerheight], color="blue")
    plt.text( bl[1], edgeheight/2, "{:.1f}".format(edgeheight), va='center', ha='right', rotation='vertical' )
    plt.text( center[1], centerheight/2, "{:.1f}".format(centerheight), va='center', ha='right', rotation='vertical' )
    
    plt.show()

interact_manual( PrintAndCalc, pixelpitch_um=i_pitch, hpixels=i_hpixels, vpixels=i_vpixels, fl_mm=i_fl, agl_m=i_agl, lookdown_deg=i_angle );   


interactive(children=(BoundedFloatText(value=10.0, description='Pixel Pitch (µm):', max=9999999.0, min=0.01, s…